In [3]:
import json
import pandas as pd

df = pd.read_csv("data.csv")


print(ds)

Dataset({
    features: ['contest', 'problem_name', 'problem_statement', 'problem_tags'],
    num_rows: 8343
})


In [4]:
# func. to extract difficulty rating from tags
def extract_difficulty(tags):
    if pd.isnull(tags):  # Handles when tags are null
        return 0
    for tag in tags.split(","):
        if "*" in tag:  # Difficulty is marked with a '*' symbol
            try:
                return int(tag.replace("*", ""))
            except ValueError:
                continue
    return 0  # Default difficulty 

In [5]:
dataset = []
# Process the dataset
dataset = []
for _, row in df.iterrows():
    new_entry = {
        "problem": row["problem_statement"],
        "tags": row["problem_tags"].split(",") if pd.notnull(row["problem_tags"]) else [],
        "difficulty": extract_difficulty(row["problem_tags"]),
        "contest": row["contest"],
        "problem_name": row["problem_name"],
    }
    dataset.append(new_entry)

print(len(dataset))
print(dataset[0])

with open("codeforces.json", "w") as f:
    json.dump(dataset, f, indent=4)

8343
{'problem': 'You are given n rectangles. The corners of rectangles have integer coordinates and their edges are parallel to the Ox and Oy axes. The rectangles may touch each other, but they do not overlap (that is, there are no points that belong to the interior of more than one rectangle). \n\nYour task is to determine if the rectangles form a square. In other words, determine if the set of points inside or on the border of at least one rectangle is precisely equal to the set of points inside or on the border of some square.\n\nThe first line contains a single integer n (1\u2009≤\u2009n\u2009≤\u20095). Next n lines contain four integers each, describing a single rectangle: x1, y1, x2, y2 (0\u2009≤\u2009x1\u2009<\u2009x2\u2009≤\u200931400,\u20090\u2009≤\u2009y1\u2009<\u2009y2\u2009≤\u200931400) — x1 and x2 are x-coordinates of the left and right edges of the rectangle, and y1 and y2 are y-coordinates of the bottom and top edges of the rectangle. \n\nNo two rectangles overlap (that